In [16]:
#---예제06---
from collections import namedtuple
from abc import ABC, abstractmethod

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    """고객, 장바구니, 프로모션(클래스 or 함수)"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            if callable(self.promotion):
                discount = self.promotion(self)
            else:
                discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


# 1. 클래스 기반 프로모션 (예제1)
class Promotion(ABC):
    """할인 전략 추상 클래스"""
    @abstractmethod
    def discount(self, order):
        pass


class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount


class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

# 2. 함수형 할인 전략 (예제3)
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


# 3. best_promo 함수 (예제4-5)
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)


# 4. 모듈 전역 탐색으로 자동 수집 (예제6)
auto_promos = [globals()[name] for name in globals()
               if name.endswith('_promo') and name != 'best_promo']


joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
    LineItem('banana', 4, 0.5),
    LineItem('apple', 10, 1.5),
    LineItem('watermelon', 5, 5.0)
]

banana_cart = [
    LineItem('banana', 30, 1.0),
    LineItem('apple', 10, 1.5)
]

long_order = [LineItem(str(i), 1, 1.0) for i in range(10)]


# 5. 실행 예제
print("--- 클래스 기반 전략 ---")
print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))
print(Order(joe, banana_cart, BulkItemPromo()))
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

print("\n--- 함수형 전략 ---")
print(Order(joe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))
print(Order(joe, banana_cart, bulk_item_promo))
print(Order(ann, long_order, large_order_promo))

print("\n--- best_promo ---")
print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

print("\n--- 자동 수집된 프로모션 리스트 ---")
print(auto_promos)

--- 클래스 기반 전략 ---
<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 45.00 due: 42.00>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>

--- 함수형 전략 ---
<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 45.00 due: 42.00>
<Order total: 10.00 due: 9.30>

--- best_promo ---
<Order total: 10.00 due: 9.30>
<Order total: 45.00 due: 42.00>
<Order total: 42.00 due: 39.90>

--- 자동 수집된 프로모션 리스트 ---
[<function fidelity_promo at 0x7fddb42d36a0>, <function bulk_item_promo at 0x7fddb4283f60>, <function large_order_promo at 0x7fddb4282fc0>]
